In [1]:
from collections import namedtuple
import itertools
import math
import enum

In [2]:
PointSet = namedtuple("PointSet", " points op val")

In [3]:
BOARD_SIZE=3

class op(enum.Enum):
    DIV = enum.auto()
    PROD = enum.auto()
    SUM = enum.auto()
    SUB = enum.auto()
    NULL = enum.auto()

sets = [
    PointSet( ((0,0), (1,0)), op.DIV, 2 ),
    PointSet( ((2,0), (2,1)), op.SUB, 2 ),
    PointSet( ((0,1), (0,2)), op.PROD, 6 ),
    PointSet( ((1,1), (1,2), (2,2)), op.SUM, 6 )

]

sets


[PointSet(points=((0, 0), (1, 0)), op=<op.DIV: 1>, val=2),
 PointSet(points=((2, 0), (2, 1)), op=<op.SUB: 4>, val=2),
 PointSet(points=((0, 1), (0, 2)), op=<op.PROD: 2>, val=6),
 PointSet(points=((1, 1), (1, 2), (2, 2)), op=<op.SUM: 3>, val=6)]

In [4]:
def get_nums():
    return range(1, BOARD_SIZE+1)

def get_factors(num, max_factor):
    return [i for i in range(1, max_factor+1) if num%i == 0]

    

In [5]:
def enum_set_solutions(ptset):
    solutions = {}
    if ptset.op == op.DIV:
        solutions = [ptset.val]
    if ptset.op == op.DIV:
        solutions = [(i, i*ptset.val) for i in range(1, int(BOARD_SIZE/ptset.val)+1)]
    if ptset.op == op.SUB:
        solutions = [(i, i+ptset.val) for i in range(1, BOARD_SIZE-ptset.val+1)]
    if ptset.op == op.PROD:
        factors = get_factors(ptset.val, BOARD_SIZE)
        combos = itertools.product(factors, repeat=len(ptset.points))
        possibles = [tuple(sorted(combo)) for combo in combos if math.prod(combo)==ptset.val]
        solutions = list(set(possibles))
    if ptset.op == op.SUM:
        possible_nums = get_nums()
        combos = itertools.product(possible_nums, repeat=len(ptset.points))
        possibles = [tuple(sorted(combo)) for combo in combos if sum(combo)==ptset.val]
        solutions = list(set(possibles))

    return solutions

In [6]:
[enum_set_solutions(s) for s in sets]

[[(1, 2)], [(1, 3)], [(2, 3)], [(1, 2, 3), (2, 2, 2)]]

In [7]:

possible_values = {}
for row in range(BOARD_SIZE):
    for col in range(BOARD_SIZE):
        possible_values[(row, col)] = set(get_nums())

possible_values


{(0, 0): {1, 2, 3},
 (0, 1): {1, 2, 3},
 (0, 2): {1, 2, 3},
 (1, 0): {1, 2, 3},
 (1, 1): {1, 2, 3},
 (1, 2): {1, 2, 3},
 (2, 0): {1, 2, 3},
 (2, 1): {1, 2, 3},
 (2, 2): {1, 2, 3}}

In [8]:
def print_cell(nums):
    if nums is None:
        return "unk"
    else:
        return "".join([str(i) if i in nums else " " for i in get_nums()])

def print_grid(values):
    for row in range(BOARD_SIZE):
        print ("|".join([print_cell(values[(row, col)]) for col in range(BOARD_SIZE)]))

print_grid(possible_values)


123|123|123
123|123|123
123|123|123


In [9]:
def do_pointset_math_logic():
    for s in sets:
        sols = enum_set_solutions(s)
        nums = set.union(*[set(sol) for sol in sols])
        for pt in s.points:
            possible_values[pt] = possible_values[pt].intersection(nums)

do_pointset_math_logic()
print_grid(possible_values)

12 | 23| 23
12 |123|123
1 3|1 3|123


In [10]:
# find pairs of pairs, use them to eliminate
# Eg the ' 23' in two cells of row 0 means that there can't be a 2 or 3 elsewhere in row 1
# TODO generalize this to columns
def do_pigeonhole_logic():
    for row in range(BOARD_SIZE):
        for size in range (1,4):
            pairs = {col: possible_values[(row, col)] for col in range(BOARD_SIZE) if len(possible_values[(row, col)])==size}
            paired_pairs = [pair for pair in pairs.values() if list(pairs.values()).count(pair)==size]
            nums = []
            if paired_pairs:
                nums = set.union(*[set(pair) for pair in paired_pairs])
                for col in range(BOARD_SIZE):
                    possibles = possible_values[(row, col)]
                    if possibles not in paired_pairs:
                        possible_values[(row, col)] = possibles - nums

    # For now copy/paste for cols
    for col in range(BOARD_SIZE):
        for size in range (1,4):
            pairs = {row: possible_values[(row, col)] for row in range(BOARD_SIZE) if len(possible_values[(row, col)])==size}
            paired_pairs = [pair for pair in pairs.values() if list(pairs.values()).count(pair)==size]
            nums = []
            if paired_pairs:
                nums = set.union(*[set(pair) for pair in paired_pairs])
                for row in range(BOARD_SIZE):
                    possibles = possible_values[(row, col)]
                    if possibles not in paired_pairs:
                        possible_values[(row, col)] = possibles - nums

do_pigeonhole_logic()
print_grid(possible_values)

1  | 23|  3
 2 |123|1 3
  3|1 3| 2 


In [11]:
# another iteration
do_pigeonhole_logic()
print_grid(possible_values)

1  | 2 |  3
 2 |  3|1  
  3|1  | 2 
